In [1]:
import pandas as pd


In [2]:
fila_inicial_fijo = 1
fila_inicial_indexado = 3

fijo = pd.read_excel('datos/precios_luz.xlsx', sheet_name='FIJO', skiprows=fila_inicial_fijo).iloc[:,2:]
indexado_energia = pd.read_excel('datos/precios_luz.xlsx', sheet_name='INDEXADO', skiprows=fila_inicial_indexado).iloc[:,1:12]
indexado_potencia = pd.read_excel('datos/precios_luz.xlsx', sheet_name='INDEXADO', skiprows=fila_inicial_indexado).iloc[:,13:24]
indexado_potencia = indexado_potencia.dropna()

nombres_columnas_energia = ['SISTEMA', 'TARIFA', 'CIA', 'MES', 'FEE', 'P1.', 'P2.', 'P3.', 'P4.', 'P5.', 'P6.']
nombres_columnas_potencia = ['SISTEMA', 'CIA', 'PRODUCTO', 'PRODUCTO_CIA', 'TARIFA', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6']
nombres_columnas_energia = [elemento.lower() for elemento in nombres_columnas_energia]
nombres_columnas_potencia = [elemento.lower() for elemento in nombres_columnas_potencia]


# Asigna los nombres de las columnas a los DataFrames
indexado_energia.columns = nombres_columnas_energia
indexado_potencia.columns = nombres_columnas_potencia


In [3]:
columns_to_convert = ['P1.', 'P2.', 'P3.', 'P4.', 'P5.', 'P6.']

for column in columns_to_convert:
    fijo[column] = pd.to_numeric(fijo[column], errors='coerce')

In [4]:
'''def modificar_producto(fila):
    if fila['cia'] == 'AEQ':
        return f"{fila['producto cia']}_fee{fila['fee']}"
    else:
        return fila['producto cia']'''

In [5]:
'''fijo['producto cia'] = fijo.apply(modificar_producto, axis=1)'''

In [6]:
'''fijo = fijo.drop('fee', axis=1)'''

In [4]:
fijo = fijo.rename(columns={'producto cia': 'producto_cia'})

In [5]:
fijo.dropna(inplace=True)
indexado_energia.dropna(inplace=True)
indexado_potencia.dropna(inplace=True)

In [6]:
fijo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 898 entries, 0 to 917
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sistema       898 non-null    object 
 1   cia           898 non-null    object 
 2   producto      898 non-null    object 
 3   producto_cia  898 non-null    object 
 4   tarifa        898 non-null    object 
 5   fee           898 non-null    object 
 6   P1            898 non-null    float64
 7   P2            898 non-null    float64
 8   P3            898 non-null    float64
 9   P4            898 non-null    float64
 10  P5            898 non-null    float64
 11  P6            898 non-null    float64
 12  P1.           898 non-null    float64
 13  P2.           898 non-null    float64
 14  P3.           898 non-null    float64
 15  P4.           898 non-null    float64
 16  P5.           898 non-null    float64
 17  P6.           898 non-null    float64
dtypes: float64(12), object(6)
memory us

In [8]:
fijo = fijo.rename(columns={'P1':'p1','P2':'p2','P3':'p3','P4':'p4','P5':'p5','P6':'p6','P1.':'p1_','P2.':'p2_','P3.':'p3_','P4.':'p4_','P5.':'p5_','P6.':'p6_'})
indexado_energia = indexado_energia.rename(columns={'p1.':'p1_','p2.':'p2_','p3.':'p3_','p4.':'p4_','p5.':'p5_','p6.':'p6_'})
indexado_potencia = indexado_potencia.rename(columns={'p1':'p1','p2':'p2','p3':'p3','p4':'p4','p5':'p5','p6':'p6'})

In [9]:
fijo.to_csv('datos/fijo.csv',index=False)
indexado_energia.to_csv('datos/indexado_energia.csv',index=False)
indexado_potencia.to_csv('datos/indexado_potencia.csv',index=False)

In [10]:
fijo.head(1)

sistema      cia producto producto_cia tarifa fee        p1       p2  \
0  PENINSULA  ACCIONA     FIJO       CIERZO  2.0TD   -  0.088121  0.01957   

    p3   p4   p5   p6       p1_       p2_       p3_  p4_  p5_  p6_  
0  0.0  0.0  0.0  0.0  0.355703  0.304626  0.265992  0.0  0.0  0.0

In [11]:
indexado_energia.head(1)

sistema tarifa      cia        mes     fee       p1_       p2_       p3_  \
0  BALEARES  2.0TD  ACCIONA 2023-06-01  Cierzo  0.212767  0.165975  0.142781   

   p4_  p5_  p6_  
0  0.0  0.0  0.0

In [12]:
indexado_potencia.head(1)

sistema      cia  producto producto_cia tarifa        p1       p2   p3  \
0  BALEARES  ACCIONA  INDEXADO       CIERZO  2.0TD  0.088121  0.01957  0.0   

    p4   p5   p6  
0  0.0  0.0  0.0

In [13]:
from sqlalchemy import create_engine
import psycopg2

user = 'postgres'
password = 'DesafioNG1'
host = '35.241.146.138'
port = '5432' 
database = 'postgres'

connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)


fijo.to_sql('precios_fijo', con=engine, if_exists='replace', index=False)


indexado_energia.to_sql('precios_index_energia', con=engine, if_exists='replace', index=False)


indexado_potencia.to_sql('precios_index_potencia', con=engine, if_exists='replace', index=False)


379